# Modules and session

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,isnan, when, count

In [2]:
spark=SparkSession.builder.appName('Translate').getOrCreate()

22/09/22 14:17:54 WARN Utils: Your hostname, LAPTOP-7NHMLFJ5 resolves to a loopback address: 127.0.1.1; using 172.31.191.161 instead (on interface eth0)
22/09/22 14:17:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/22 14:17:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Data

In [3]:
data=spark.read.option('header', 'true').csv('files.csv',inferSchema= True)

In [4]:
data.show(5)

+--------------------+------------+--------------------+
|        wav_filename|wav_filesize|          transcript|
+--------------------+------------+--------------------+
|audios/4da6b70e-0...|      219064|y aquí en dos pal...|
|audios/8c2ab30b-0...|      271910|cuando los consej...|
|audios/ca73c951-c...|       64520|su mujer con la c...|
|audios/067c4606-7...|       84222|y otros que se po...|
|audios/49a08f90-3...|       77316|al oír mis pasos ...|
+--------------------+------------+--------------------+
only showing top 5 rows



# Prepossessing

## Missings

In [5]:
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+------------+------------+----------+
|wav_filename|wav_filesize|transcript|
+------------+------------+----------+
|           0|           0|         0|
+------------+------------+----------+



In [6]:
data.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in data.columns]).show()

+------------+------------+----------+
|wav_filename|wav_filesize|transcript|
+------------+------------+----------+
|           0|           0|         0|
+------------+------------+----------+



## Traducción

In [12]:
rdd= spark.sparkContext.textFile("files.csv")

In [20]:
spark.sql.sh

AttributeError: module 'pyspark.sql' has no attribute 'shuffle'

In [16]:
rdd.map(lambda x: x.split(",")).collect()

22/09/22 14:33:28 ERROR Executor: Exception in task 0.0 in stage 13.0 (TID 22)
org.apache.spark.SparkException: 
Error from python worker:
  Traceback (most recent call last):
    File "/usr/local/lib/python3.10/runpy.py", line 187, in _run_module_as_main
      mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
    File "/usr/local/lib/python3.10/runpy.py", line 110, in _get_module_details
      __import__(pkg_name)
    File "/home/bluterplay/.local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/__init__.py", line 144, in <module>
    File "/home/bluterplay/.local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/__init__.py", line 42, in <module>
    File "/home/bluterplay/.local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/types.py", line 28, in <module>
      
    File "/usr/local/lib/python3.10/ctypes/__init__.py", line 8, in <module>
      from _ctypes import Union, Structure, Array
  ModuleNotFoundEr

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 13.0 failed 1 times, most recent failure: Lost task 0.0 in stage 13.0 (TID 22) (172.31.191.161 executor driver): org.apache.spark.SparkException: 
Error from python worker:
  Traceback (most recent call last):
    File "/usr/local/lib/python3.10/runpy.py", line 187, in _run_module_as_main
      mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
    File "/usr/local/lib/python3.10/runpy.py", line 110, in _get_module_details
      __import__(pkg_name)
    File "/home/bluterplay/.local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/__init__.py", line 144, in <module>
    File "/home/bluterplay/.local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/__init__.py", line 42, in <module>
    File "/home/bluterplay/.local/lib/python3.8/site-packages/pysp

22/09/22 14:33:29 WARN TaskSetManager: Lost task 1.0 in stage 13.0 (TID 23) (172.31.191.161 executor driver): TaskKilled (Stage cancelled)


In [42]:
i=0
for row in rdd.collect():
    i+=1
    if(i>10):
        break
    print(row)

Row(wav_filename='audios/4da6b70e-0108-4f75-80ae-3d71f1dd2c2b.wav', wav_filesize=219064, transcript='y aquí en dos palotadas hemos encontrado robustas columnas donde apoyar la grandiosa fábrica de su alcurnia')
Row(wav_filename='audios/8c2ab30b-0fd4-41c3-9724-3b15f2ee2c27.wav', wav_filesize=271910, transcript='cuando los consejeros escucharon aquello quedaron estremecidos y se dijeron dios ha prohibido que padres se casen con sus hijas')
Row(wav_filename='audios/ca73c951-c62a-41fe-a953-9871514151f2.wav', wav_filesize=64520, transcript='su mujer con la cara entre las manos')
Row(wav_filename='audios/067c4606-777b-4fb2-bc6f-8185fbec9016.wav', wav_filesize=84222, transcript='y otros que se podían echar a la oreja de un toro')
Row(wav_filename='audios/49a08f90-3fc0-43ad-bd5e-a6b671cafdd4.wav', wav_filesize=77316, transcript='al oír mis pasos alzó la cabeza')
Row(wav_filename='audios/f97f3d33-af8a-4f5b-a939-2a0153a8566a.wav', wav_filesize=223566, transcript='resplandeciente con un sinnúmero

In [45]:
rdd.map(lambda x: (x[0],1)).collect()

22/09/22 14:15:38 ERROR Executor: Exception in task 0.0 in stage 29.0 (TID 73)
org.apache.spark.SparkException: 
Error from python worker:
  Traceback (most recent call last):
    File "/usr/local/lib/python3.10/runpy.py", line 187, in _run_module_as_main
      mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
    File "/usr/local/lib/python3.10/runpy.py", line 110, in _get_module_details
      __import__(pkg_name)
    File "/home/bluterplay/.local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/__init__.py", line 144, in <module>
    File "/home/bluterplay/.local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/__init__.py", line 42, in <module>
    File "/home/bluterplay/.local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/types.py", line 28, in <module>
      
    File "/usr/local/lib/python3.10/ctypes/__init__.py", line 8, in <module>
      from _ctypes import Union, Structure, Array
  ModuleNotFoundEr

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 29.0 failed 1 times, most recent failure: Lost task 0.0 in stage 29.0 (TID 73) (172.31.191.161 executor driver): org.apache.spark.SparkException: 
Error from python worker:
  Traceback (most recent call last):
    File "/usr/local/lib/python3.10/runpy.py", line 187, in _run_module_as_main
      mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
    File "/usr/local/lib/python3.10/runpy.py", line 110, in _get_module_details
      __import__(pkg_name)
    File "/home/bluterplay/.local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/__init__.py", line 144, in <module>
    File "/home/bluterplay/.local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/__init__.py", line 42, in <module>
    File "/home/bluterplay/.local/lib/python3.8/site-packages/pysp

22/09/22 14:15:39 WARN TaskSetManager: Lost task 3.0 in stage 29.0 (TID 76) (172.31.191.161 executor driver): TaskKilled (Stage cancelled)


22/09/22 14:15:39 WARN TaskSetManager: Lost task 2.0 in stage 29.0 (TID 75) (172.31.191.161 executor driver): TaskKilled (Stage cancelled)


22/09/22 14:15:40 WARN TaskSetManager: Lost task 1.0 in stage 29.0 (TID 74) (172.31.191.161 executor driver): TaskKilled (Stage cancelled)


In [21]:
a= [('hola',2), ('Como', 2), ("tres", 1)]
rdd=spark.sparkContext.parallelize(a)

In [25]:
rdd.map(lambda x: (x[0],x[1]+1)).collect()

22/09/22 14:40:13 ERROR Executor: Exception in task 5.0 in stage 17.0 (TID 46)
org.apache.spark.SparkException: 
Error from python worker:
  Traceback (most recent call last):
    File "/usr/local/lib/python3.10/runpy.py", line 187, in _run_module_as_main
      mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
    File "/usr/local/lib/python3.10/runpy.py", line 110, in _get_module_details
      __import__(pkg_name)
    File "/home/bluterplay/.local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/__init__.py", line 144, in <module>
    File "/home/bluterplay/.local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/__init__.py", line 42, in <module>
    File "/home/bluterplay/.local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/types.py", line 28, in <module>
      
    File "/usr/local/lib/python3.10/ctypes/__init__.py", line 8, in <module>
      from _ctypes import Union, Structure, Array
  ModuleNotFoundEr

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 17.0 failed 1 times, most recent failure: Lost task 5.0 in stage 17.0 (TID 46) (172.31.191.161 executor driver): org.apache.spark.SparkException: 
Error from python worker:
  Traceback (most recent call last):
    File "/usr/local/lib/python3.10/runpy.py", line 187, in _run_module_as_main
      mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
    File "/usr/local/lib/python3.10/runpy.py", line 110, in _get_module_details
      __import__(pkg_name)
    File "/home/bluterplay/.local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/__init__.py", line 144, in <module>
    File "/home/bluterplay/.local/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/__init__.py", line 42, in <module>
    File "/home/bluterplay/.local/lib/python3.8/site-packages/pysp

22/09/22 14:40:13 WARN TaskSetManager: Lost task 3.0 in stage 17.0 (TID 44) (172.31.191.161 executor driver): TaskKilled (Stage cancelled)


22/09/22 14:40:14 WARN TaskSetManager: Lost task 1.0 in stage 17.0 (TID 42) (172.31.191.161 executor driver): TaskKilled (Stage cancelled)


22/09/22 14:40:15 WARN TaskSetManager: Lost task 4.0 in stage 17.0 (TID 45) (172.31.191.161 executor driver): TaskKilled (Stage cancelled)


22/09/22 14:40:15 WARN TaskSetManager: Lost task 0.0 in stage 17.0 (TID 41) (172.31.191.161 executor driver): TaskKilled (Stage cancelled)


22/09/22 14:40:16 WARN TaskSetManager: Lost task 6.0 in stage 17.0 (TID 47) (172.31.191.161 executor driver): TaskKilled (Stage cancelled)


22/09/22 14:40:17 WARN TaskSetManager: Lost task 2.0 in stage 17.0 (TID 43) (172.31.191.161 executor driver): TaskKilled (Stage cancelled)


22/09/22 14:40:18 WARN TaskSetManager: Lost task 7.0 in stage 17.0 (TID 48) (172.31.191.161 executor driver): TaskKilled (Stage cancelled)
